In [75]:
import os
from fastapi import FastAPI
import json
import torch
# from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from langchain_neo4j import Neo4jGraph
from langchain_neo4j import Neo4jVector

from langchain_experimental.graph_transformers import LLMGraphTransformer

from langchain_core.runnables import RunnablePassthrough
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser

from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain.prompts import ChatPromptTemplate

In [76]:
use_cuda = torch.cuda.is_available()
if use_cuda:
    print("CUDA is available! Using GPU.")
    ollama_backend = "cuda"
else:
    print("CUDA is not available. Using CPU.")
    ollama_backend = "cpu"

# 1️⃣ CONNECT TO NEO4J DATABASE
graph = Neo4jGraph(
    url="bolt://localhost:7689",
    username="neo4j",
    password="password",
    refresh_schema=False
)

CUDA is available! Using GPU.


In [77]:
# 2️⃣ Function to ingest tourist guides into Neo4j
def ingest_guide(text):
    """Extracts entities from a tourist guide and stores them in Neo4j."""
    
    documents = [Document(page_content=text)]
    llm = ChatOllama(model="mistral", temperature=0, backend=ollama_backend)
    graph_transformer = LLMGraphTransformer(llm=llm)
    graph_documents = graph_transformer.convert_to_graph_documents(documents)
    graph.add_graph_documents(graph_documents, baseEntityLabel=True, include_source=True)

    # Use the correct import and remove backend parameter if it is not supported
    embed = OllamaEmbeddings(model="mxbai-embed-large")
    vector_index = Neo4jVector.from_existing_graph(
        embedding=embed,
        url="bolt://localhost:7689",
        username="neo4j",
        password="password",
        search_type="hybrid",
        node_label="Document",
        text_node_properties=["text"],
        embedding_node_property="embedding"
    )
    
    global vector_retriever
    vector_retriever = vector_index.as_retriever()

In [84]:
# 3️⃣ FUNCTION TO QUERY NEO4J
def query_neo4j(question):
    """Extracts entities from a question and retrieves relationships from Neo4j."""

    class Entities(BaseModel):
        names: list[str] = Field(..., description="Extracted entities from text")

    # Define prompt
    prompt = ChatPromptTemplate.from_messages([
        ("system", "Extract locations, events, foods, and attractions from the question."),
        ("human", "{question}")
    ])

    # Use ChatOllama
    llm = ChatOllama(model="mistral", format="json", temperature=0, backend=ollama_backend)

    # Invoke model with structured output
    response = llm.invoke(prompt.format(question=question))
    print("📝 Raw LLM Response:", response.content)

    # Ensure response is valid
    if not response or not response.content:
        print("⚠️ No response from Ollama!")
        return ""

    try:
        # Parse JSON response
        parsed_response = json.loads(response.content)
        entities = parsed_response.get("names", [])
    except json.JSONDecodeError:
        print(f"❌ Failed to parse JSON: {response.content}")
        return ""

    print("✅ Retrieved Entities:", entities)
    
    result = []
    for entity in entities:
        query_response = graph.query(
            """
            CALL {
                WITH $entity AS entity
                MATCH (p:Place {id: entity})-[:*]->(e)
                RETURN p.id AS source_id, type(r) AS relationship, e.id AS target_id
                LIMIT 50
            }
            RETURN source_id, relationship, target_id
            """,
            {"entity": entity},
        )

        # Handle empty results
        if not query_response:
            result.append(f"No relationships found for entity: {entity}")
        else:
            result.extend([f"{el['source_id']} - {el['relationship']} -> {el['target_id']}" for el in query_response])

    return "\n".join(result)

In [79]:
# 4️⃣ FUNCTION FOR HYBRID SEARCH (Neo4j Graph + Vector Embeddings)
def query_tourist_guide(question):
    """Performs hybrid search using Neo4j graph data and vector embeddings."""

    def full_retriever(question: str):
        graph_data = query_neo4j(question)
        
        vector_query = """
        CALL {
            WITH $query AS query
            CALL db.index.vector.queryNodes($index, $k, $embedding) 
            YIELD node, score 
            WITH collect({node: node, score: score}) AS nodes, max(score) AS max 
            UNWIND nodes AS n 
            RETURN n.node AS node, (n.score / max) AS score
        }
        RETURN node.text AS text, node.metadata AS metadata, score
        """

        vector_data = [el.page_content for el in vector_retriever.invoke(question)]
        
        return f"Graph data: {graph_data}\nVector data: {'#Document '.join(vector_data)}"

    template = """Answer the question based only on the following context:
    {context}
    Question: {question}
    Answer:"""
    
    prompt = ChatPromptTemplate.from_template(template)
    llm = ChatOllama(model="llama3", temperature=0, backend=ollama_backend)
    
    chain = prompt | llm | StrOutputParser()
    return chain.invoke({"context": full_retriever(question), "question": question})

In [80]:
# Example tourist guide data
guide_text = """
Babina Greda is probably the best kept secret of eastern Croatia.
The valleys are greener, the traditional kulen even more delicious, and the girls have the prettiest smiles.
The Slavonian treasure chest will surprise you wherever you go.

Gatherings of Stanari" (Stanarski susreti)
End of August (beginning of September) is reserved for the "Gatherings of Stanari".
This event revives memories of life on traditional farms called "stanovi".
Visitors enjoy competitions in farming skills, cooking duels, and traditional dish preparation.
"""

# Ingest guide data into Neo4j
print("Ingesting tourist guide into Neo4j...")
ingest_guide(guide_text)

Ingesting tourist guide into Neo4j...


In [85]:
# Example queries
queries = [
    "What is Babina Greda known for?",
    "What traditional events happen in Slavonia?",
    "Tell me about food in Dalmatia."
]

for q in queries:
    print(f"\nQuerying: {q}\n")
    response = query_tourist_guide(q)
    print("Final Answer:\n", response)



Querying: What is Babina Greda known for?

📝 Raw LLM Response: {
    "locations": ["Babina Greda"],
    "events": [],
    "foods": [],
    "attractions": []
}
✅ Retrieved Entities: []
Final Answer:
 According to the provided text, Babina Greda is probably the best kept secret of eastern Croatia. It is described as having greener valleys, more delicious traditional kulen (a type of sausage), and prettier smiles from the girls. Additionally, it is mentioned that the Slavonian treasure chest will surprise visitors wherever they go.

Querying: What traditional events happen in Slavonia?

📝 Raw LLM Response: {
  "locations": ["Slavonia"],
  "events": ["traditional events"]
}
✅ Retrieved Entities: []
Final Answer:
 According to the provided text, there is no specific mention of traditional events happening in Slavonia. However, it does mention that Babina Greda is referred to as the "Slavonian treasure chest" and that the region has valleys that are greener, kulen that is more delicious, an